# Regressão Linear - Implantação

Este componente realiza predições usando um modelo de Regressão Linear usando [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html). <br>
Scikit-learn é uma biblioteca open source de machine learning que suporta apredizado supervisionado e não supervisionado. Também provê várias ferramentas para ajustes de modelos, pré-processamento de dados, seleção e avaliação de modelos, além de outras funcionalidades.

Este notebook apresenta:
- como carregar modelos e outros resultados do treinamento.
- como utilizar um modelo para fornecer predições em tempo real.

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [ ]:
%%writefile Model.py
import joblib
import numpy as np
import pandas as pd


class Model(object):
    def __init__(self):
        # Carrega artefatos: estimador, etc
        artifacts = joblib.load("/tmp/data/linear-regression.joblib")
        self.pipeline = artifacts["pipeline"]
        self.features_names_training = artifacts["columns"]
        self.columns_to_filter = artifacts["columns_to_filter"]
        self.new_columns = artifacts["new_columns"]
        self.features_after_pipeline = artifacts["features_after_pipeline"]

    def class_names(self):
        column_names = np.concatenate(
            (self.features_after_pipeline, [self.new_columns])
        )
        return column_names.tolist()

    def predict(self, X, feature_names, meta=None):
        if feature_names:
            # Antes de utilizar o conjunto de dados X no modelo, reordena suas features de acordo com a ordem utilizada no treinamento
            df = pd.DataFrame(X, columns=feature_names)
            X = df[self.columns_to_filter].to_numpy()

        # Realiza a predição
        y_pred = self.pipeline.predict(X)

        # adiciona a predição ao conjunto de dados
        df[self.new_columns] = y_pred

        return df.to_numpy()

## Teste do serviço REST

Crie um arquivo `contract.json` com os seguintes atributos:

- `features` : A lista de features em uma requisição.
- `targets` : A lista de valores retornados pelo método `predict`.

Cada `feature` pode conter as seguintes informações:

- `name` : nome da feature
- `ftype` : tipo da feature : **continuous** ou **categorical**
- `dtype` : tipo de dado : **FLOAT** ou **INT** : *obrigatório para ftype continuous*
- `range` : intervalo de valores numéricos : *obrigatório para ftype continuous*
- `values` : lista de valores categóricos : *obrigatório para ftype categorical*

Em seguida, utilize a função `test_deployment` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para simular predição em tempo-real.<br>

In [ ]:
%%writefile contract.json
{
    "features": [
        {
            "name": "crim",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        },
        {
            "name": "zn",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        },
        {
            "name": "indus",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        },
        {
            "name": "chas",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 1.0]
        },
        {
            "name": "nox",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 1.0]
        },
        {
            "name": "rm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 10.0]
        },
        {
            "name": "age",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        },
        {
            "name": "dis",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        },
        {
            "name": "rad",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [1.0, 24.0]
        },
        {
            "name": "tax",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 1000.0]
        },
        {
            "name": "ptratio",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        },
        {
            "name": "black",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 1000.0]
        },
        {
            "name": "lstat",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        }
    ],
    "targets": [
        {
            "name": "estimate",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        }
    ]
}

In [ ]:
from platiagro.deployment import test_deployment

test_deployment("contract.json")